In [30]:
from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama
from os import getenv
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model="meta-llama/llama-3.2-3b-instruct:free",
    temperature=0.5,
    # max_tokens=512,

)
# llm = ChatOllama(model='qwen2.5:0.5b', temperature=0.5, max_tokens=512)

result = llm.invoke("Hello, how are you today?")
result.content

"I'm just a language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you have. How can I assist you today?"

# JSON Output Parsing

In [31]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

{'setup': 'Why did the scarecrow win an award?',
 'punchline': 'Because he was outstanding in his field!'}

In [32]:
parser.get_format_instructions()


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}\n```'

## Streaming Output

In [33]:
for s in chain.stream({"query": joke_query}):
    print(s)

{}
{'setup': "Why don't skeletons fight each other?"}
{'setup': "Why don't skeletons fight each other?", 'punchline': "They don't have"}
{'setup': "Why don't skeletons fight each other?", 'punchline': "They don't have the guts."}


In [34]:
joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

{'joke': "Why couldn't the bicycle stand up by itself? Because it was two-tired!"}

# XML

In [35]:
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate

actor_query = "Generate the shortened filmography for Tom Hanks."

output = llm.invoke(
    f"""{actor_query}
Please enclose the movies in <movie></movie> tags"""
)

print(output.content)

Here's a shortened filmography for Tom Hanks:

- <movie>He Knows You're Alone</movie> (1980)
- <movie>Splash</movie> (1984)
- <movie>Big</movie> (1988)
- <movie>Turner & Hooch</movie> (1989)
- <movie>The Bonfire of the Vanities</movie> (1990)
- <movie>A League of Their Own</movie> (1992)
- <movie>Sleepless in Seattle</movie> (1993)
- <movie>Philadelphia</movie> (1993)
- <movie>Forrest Gump</movie> (1994)
- <movie>Apollo 13</movie> (1995)
- <movie>Toy Story</movie> (1995)
- <movie>That Thing You Do!</movie> (1996)
- <movie>Saving Private Ryan</movie> (1998)
- <movie>You've Got Mail</movie> (1998)
- <movie>The Green Mile</movie> (1999)
- <movie>Cast Away</movie> (2000)
- <movie>Road to Perdition</movie> (2002)
- <movie>The Terminal</movie> (2004)
- <movie>The Polar Express</movie> (2004)
- <movie>Charlie Wilson's War</movie> (2007)
- <movie>Angels & Demons</movie> (2009)
- <movie>Toy Story 3</movie> (2010)
- <movie>Larry Crowne</movie> (2011)
- <movie>Extremely Loud & Incredibly Close</m

In [46]:
parser = XMLOutputParser()

# We will add these instructions to the prompt below
print(parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


In [45]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": actor_query})

OutputParserException: Failed to parse XML format from completion <movies>
    <movie>
        <film>
            <name>Forrest Gump</name>
            <genre>Comedy, Drama</genre>
        </film>
    </movie>
    <movie>
        <film>
            <name>The Da Vinci Code</name>
            <genre>Adventure, Mystery, Thriller</genre>
        </film>
    </movie>
    <movie>
        <film>
            <name>Captain Phillips</name>
            <genre>Biography, Drama, Thriller</genre>
        </film>
    </movie>
    <movie>
        <film>
            <name>Sully</name>
            <genre>Biography, Drama, Thriller</genre>
        </film>
    </movie>
    <movie>
        <film>
            <name>Cast Away</name>
            <genre>Adventure, Drama, Thriller</genre>
        </film>
    </movie>
    <movie>
        <film>
            <name>The Polar Express</name>
            <genre>Adventure, Family, Fantasy</genre>
        </film>
    </movie>
    <movie>
        <film>
            <name

In [44]:
parser = XMLOutputParser(tags=["movies", "actor", "film", "name", "genre"])

# We will add these instructions to the prompt below
print(parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
['movies', 'actor', 'film', 'name', 'genre']
```


In [39]:
for s in chain.stream({"query": actor_query}):
    print(s)

{'filmography': [{'actor': [{'name': 'Tom Hanks'}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'title': 'Forrest Gump'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'year': '1994'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'title': 'Cast Away'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'year': '2000'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'title': 'Apollo 13'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'year': '1995'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'title': 'Sully'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'year': '2016'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'title': 'Saving Private Ryan'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'year': '1998'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'title': 'Big'}]}]}]}]}
{'filmography': [{'actor': [{'movies': [{'movie': [{'year': '1988'}]}]}

# CSV

# Ooutput Fixing

# Retry with Error Handling

# Pyndatic Models

# Yaml

In [40]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

Joke(setup="Why couldn't the bicycle stand up by itself?", punchline='Because it was two-tired.')

In [43]:
r = parser.get_format_instructions()
print(r)

The output should be formatted as a YAML instance that conforms to the given JSON schema below.

# Examples
## Schema
```
{"title": "Players", "description": "A list of players", "type": "array", "items": {"$ref": "#/definitions/Player"}, "definitions": {"Player": {"title": "Player", "type": "object", "properties": {"name": {"title": "Name", "description": "Player name", "type": "string"}, "avg": {"title": "Avg", "description": "Batting average", "type": "number"}}, "required": ["name", "avg"]}}}
```
## Well formatted instance
```
- name: John Doe
  avg: 0.3
- name: Jane Maxfield
  avg: 1.4
```

## Schema
```
{"properties": {"habit": { "description": "A common daily habit", "type": "string" }, "sustainable_alternative": { "description": "An environmentally friendly alternative to the habit", "type": "string"}}, "required": ["habit", "sustainable_alternative"]}
```
## Well formatted instance
```
habit: Using disposable water bottles for daily hydration.
sustainable_alternative: Switch t

setup="Why couldn't the bicycle stand up by itself?" punchline='Because it was two-tired.'


# Pandas Dataframe

# Enum

# Date and Time

# Structured